In [1]:
!which python

/Users/ahmadabdullahtariq/Documents/Projects/Thesis/.venv/bin/python


In [1]:
import pandas as pd
import numpy as np 
import plotly.graph_objects as go

In [2]:
df = pd.read_pickle('../dataset/preprocessed/284_imputed.pkl')

In [3]:
print(df.isna().sum())

df['Definitive'] = df['Definitive'].astype('int')
df['Rooms'] = df['Rooms'].astype('int')

print(df.dtypes)

TargetDate      0
Definitive      0
Rooms           0
SnapshotDate    0
LAR             0
dtype: int64
TargetDate      datetime64[ns]
Definitive               int64
Rooms                    int64
SnapshotDate    datetime64[ns]
LAR                    float64
dtype: object


# Linear Regresison Model
## Considering final Data Bookings

In [4]:
# consider data for 2018 and 2019
df = df[df['TargetDate']<='2019-12-31']
# creating new columns
df['LeadTime'] = (df['TargetDate']-df['SnapshotDate']).dt.days
df['DOW'] = df.TargetDate.dt.dayofweek
df = df.drop(columns=['LAR','SnapshotDate'])
# just rearrange columns 
df = df[['TargetDate','DOW','LeadTime','Rooms','Definitive']]

In [5]:
def curve(df):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.LeadTime, y=df.Definitive,mode='markers',name='Price'))
    fig.show()   

def days_of_week(df):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.DOW, y=df.Definitive,mode='markers',name='Price'))
    fig.show()   

In [111]:
curve(df[(df['DOW']==2)&(df['TargetDate']>='2019-11-01')])
days_of_week(df[(df['TargetDate']>='2019-09-01')])

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

In [7]:
# import numpy as np
# from sklearn.model_selection import KFold

# mse = []
# kf = KFold(n_splits=100)
# for degree in range(0,10):
#     for train, test in kf.split(X.values):
#         # print("%s %s" % (train, test))
#         X_train = X.iloc[train].values
#         X_test = X.iloc[test].values

#         y_train = y.iloc[train].values
#         y_test = y.iloc[test].values
#         # print(y_train)
#         # print(X_train)
#         model = polynomialRegression(degree).fit(X_train, y_train)
#         y_pred = model.predict(X_test)
#         mse.append(mean_squared_error(y_test,y_pred))
#         # print(mse)
#     print("Degree : {}\t MSE : {}".format(degree, np.sqrt(mse).mean()))

In [8]:
df[df['Rooms']==0]

,TargetDate,DOW,LeadTime,Rooms,Definitive
50423,2019-08-28,2,0,0,274
50507,2019-08-29,3,1,0,266
50591,2019-08-30,4,2,0,266
50675,2019-08-31,5,3,0,267
50758,2019-09-01,6,4,0,223
...,...,...,...,...,...
57711,2019-11-21,3,85,0,31
57798,2019-11-22,4,86,0,30
57886,2019-11-23,5,87,0,29
57975,2019-11-24,6,88,0,19


In [9]:
df['Rooms'] = df['Rooms'].replace(0, np.nan)
df['Rooms'] = df['Rooms'].ffill().bfill()

In [12]:
df['Occupancy'] = df['Definitive']/df['Rooms']

In [14]:
index = int(len(df)*0.75)
train = df[:index]
test = df[index:]

train.pop('Rooms')
test.pop('Rooms')
# train.pop('DOW')
# test.pop('DOW')
train.pop('Definitive')
test.pop('Definitive')
dates_train = train.pop('TargetDate')
dates_test = test.pop('TargetDate')

y_train = train.pop('Occupancy')
y_test = test.pop('Occupancy')
X_train = train
X_test = test

In [15]:
def polynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree),LinearRegression(**kwargs))

In [16]:
model = polynomialRegression(7).fit(X_train, y_train)


In [17]:
y_pred = model.predict(X_test)

In [18]:
mean_squared_error(y_test,y_pred)

0.019520608940609807

In [19]:
X_test['True'] = y_test
X_test['Pred'] = y_pred
X_test['TargetDate'] = dates_test

In [22]:
temp = X_test[X_test['TargetDate']=='2019-09-27']
fig = go.Figure()
fig.add_trace(go.Scatter(x=temp['LeadTime'], y=temp['True'],mode='markers',name='True'))
fig.add_trace(go.Scatter(x=temp['LeadTime'], y=temp['Pred'],mode='markers',name='Pred'))
fig.show()   

## Considering 90 Days Development of Bookings

In [73]:
df = pd.read_pickle('../dataset/preprocessed/284_imputed.pkl')

In [74]:
print(df.isna().sum())

df['Definitive'] = df['Definitive'].astype('int')
df['Rooms'] = df['Rooms'].astype('int')

print(df.dtypes)

TargetDate      0
Definitive      0
Rooms           0
SnapshotDate    0
LAR             0
dtype: int64
TargetDate      datetime64[ns]
Definitive               int64
Rooms                    int64
SnapshotDate    datetime64[ns]
LAR                    float64
dtype: object


In [75]:
# consider data for 2018 and 2019
df = df[df['TargetDate']<='2019-12-31']

In [76]:
df['LeadTime'] = (df['TargetDate']-df['SnapshotDate']).dt.days
# df['DOW'] = df.TargetDate.dt.dayofweek
df = df.drop(columns=['LAR','SnapshotDate'])
# just rearrange columns 
df = df[['TargetDate','LeadTime','Rooms','Definitive']]

In [77]:
df = df.pivot(index='TargetDate',columns='LeadTime',values='Definitive')
df = df.ffill(axis=1).bfill(axis=1)

In [78]:
df.head()

LeadTime,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
TargetDate,,,,,,,,,,,,,,,,,,,,,
2018-01-01,209.0,206.0,206.0,198.0,204.0,198.0,196.0,199.0,197.0,189.0,...,62.0,63.0,63.0,63.0,63.0,58.0,58.0,58.0,58.0,58.0
2018-01-02,223.0,214.0,209.0,209.0,201.0,203.0,188.0,183.0,184.0,185.0,...,52.0,50.0,51.0,51.0,51.0,51.0,49.0,49.0,48.0,48.0
2018-01-03,238.0,218.0,212.0,207.0,207.0,194.0,195.0,174.0,168.0,169.0,...,40.0,40.0,39.0,39.0,39.0,39.0,39.0,36.0,36.0,36.0
2018-01-04,234.0,222.0,211.0,208.0,203.0,203.0,189.0,186.0,161.0,152.0,...,35.0,33.0,33.0,33.0,33.0,33.0,33.0,33.0,32.0,32.0
2018-01-05,261.0,238.0,223.0,212.0,210.0,207.0,207.0,194.0,185.0,158.0,...,29.0,29.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,26.0


In [79]:
# !pip install sklearn

In [80]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

In [81]:
df.head()

LeadTime,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
TargetDate,,,,,,,,,,,,,,,,,,,,,
2018-01-01,209.0,206.0,206.0,198.0,204.0,198.0,196.0,199.0,197.0,189.0,...,62.0,63.0,63.0,63.0,63.0,58.0,58.0,58.0,58.0,58.0
2018-01-02,223.0,214.0,209.0,209.0,201.0,203.0,188.0,183.0,184.0,185.0,...,52.0,50.0,51.0,51.0,51.0,51.0,49.0,49.0,48.0,48.0
2018-01-03,238.0,218.0,212.0,207.0,207.0,194.0,195.0,174.0,168.0,169.0,...,40.0,40.0,39.0,39.0,39.0,39.0,39.0,36.0,36.0,36.0
2018-01-04,234.0,222.0,211.0,208.0,203.0,203.0,189.0,186.0,161.0,152.0,...,35.0,33.0,33.0,33.0,33.0,33.0,33.0,33.0,32.0,32.0
2018-01-05,261.0,238.0,223.0,212.0,210.0,207.0,207.0,194.0,185.0,158.0,...,29.0,29.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,26.0


In [82]:
index = int(len(df)*0.75)
train = df.iloc[:index]
test = df.iloc[index:]

y_test = test.pop(0)
y_train = train.pop(0)

X_test = test
X_train = train

In [83]:
X = X.reset_index(drop=True)

In [84]:
def polynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree),LinearRegression(**kwargs))

In [85]:
model = polynomialRegression(3).fit(X_train, y_train)

In [86]:
y_pred = model.predict(X_test)

In [87]:
mean_squared_error(y_test,y_pred)

2986.486062342283

In [88]:
X_test['True'] = y_test
X_test['Pred'] = y_pred

In [89]:
# temp = X_test[X_test['TargetDate']=='2019-09-27']
fig = go.Figure()
fig.add_trace(go.Scatter(x=X_test.index, y=X_test['True'],mode='markers',name='True'))
fig.add_trace(go.Scatter(x=X_test.index, y=X_test['Pred'],mode='markers',name='Pred'))
fig.show()   

In [118]:
import numpy as np
from sklearn.model_selection import KFold

mse = []
kf = KFold(n_splits=100)
for degree in range(0,10):
    for train, test in kf.split(X.values):
        # print("%s %s" % (train, test))
        X_train = X.iloc[train].values
        X_test = X.iloc[test].values

        y_train = y.iloc[train].values
        y_test = y.iloc[test].values
        # print(y_train)
        # print(X_train)
        model = polynomialRegression(degree).fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mse.append(mean_squared_error(y_test,y_pred))
        # print(mse)
    print("Degree : {}\t MSE : {}".format(degree, np.sqrt(mse).mean()))

Degree : 0	 MSE : 44.99850001084681
Degree : 1	 MSE : 44.998500010846804
Degree : 2	 MSE : 44.998500010846804
Degree : 3	 MSE : 44.998500010846804
Degree : 4	 MSE : 44.998500010846804


KeyboardInterrupt: 